# 🎯 CRP Scanner Demo Notebook

This notebook demonstrates the CRP (Close, Range, Pattern) Scanner for intraday stock selection.

## What is CRP?
- **C**lose: Stocks closing near their daily high or low
- **R**ange: Trading ranges that are narrow/tight
- **P**attern: Combines volume, momentum, and technical indicators

## Features
- Date range analysis with end-of-day tracking
- Probability scoring system (0-100)
- Professional table display
- CSV export functionality
- Performance metrics and success rate analysis

In [1]:
# Setup and Imports
import sys
import os
from pathlib import Path
from datetime import date, time, timedelta
import pandas as pd

# Add project root to Python path
current_dir = Path.cwd()
if current_dir.name == 'scanner':
    project_root = current_dir.parent.parent
elif current_dir.name == 'notebook':
    project_root = current_dir.parent
else:
    project_root = current_dir

sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

# Import CRP Scanner
from src.application.scanners.strategies.crp_scanner import CRPScanner

print(f"Project root: {project_root}")
print("✅ Imports successful!")

INFO:src.infrastructure.config.settings:Database file 'data/duckdb/bars.duckdb' does not exist yet - will be created on first use


Project root: /Users/apple/Downloads/duckDbData
✅ Imports successful!


In [2]:
# Initialize CRP Scanner
# Initialize CRP Scanner with proper port injection
from src.app.startup import get_scanner
scanner = get_scanner('crp')

# Display configuration
print("🎯 CRP Scanner Configuration:")
print(f"   Close Threshold: {scanner.config['close_threshold_pct']}%")
print(f"   Range Threshold: {scanner.config['range_threshold_pct']}%")
print(f"   Min Volume: {scanner.config['min_volume']:,}")
print(f"   Price Range: ₹{scanner.config['min_price']} - ₹{scanner.config['max_price']}")
print(f"   Max Results Per Day: {scanner.config['max_results_per_day']}")
print(f"   CRP Detection Time: {scanner.config['crp_cutoff_time']}")
print(f"   End of Day Time: {scanner.config['end_of_day_time']}")

[DEBUG] ScannerFactory.SCANNER_REGISTRY initialized: dict_keys(['breakout', 'enhanced_breakout', 'crp', 'enhanced_crp', 'technical', 'nifty500_filter', 'relative_volume', 'simple_breakout'])


ImportError: attempted relative import beyond top-level package

In [ ]:
# Single Day CRP Scan (for testing)
scan_date = date.today() - timedelta(days=1)  # Use yesterday for demo
cutoff_time = time(9, 50)

print(f"📅 Single Day Scan: {scan_date}")
print(f"⏰ Cutoff Time: {cutoff_time}")

# Perform single day scan
single_day_results = scanner.scan(scan_date, cutoff_time)

if not single_day_results.empty:
    print(f"✅ Found {len(single_day_results)} CRP patterns")
    print("\n📊 Top CRP Results:")
    print(single_day_results.head().to_string(index=False))
else:
    print("⚠️  No CRP patterns found for single day scan")

In [ ]:
# Date Range CRP Analysis
print("📈 Performing Date Range CRP Analysis...")

# Define date range (adjust as needed)
start_date = scan_date - timedelta(days=4)  # 5-day window
end_date = scan_date

print(f"   Date Range: {start_date} to {end_date}")

# Perform date range analysis
range_results = scanner.scan_date_range(
    start_date=start_date,
    end_date=end_date,
    cutoff_time=time(9, 50),
    end_of_day_time=time(15, 15)
)

if range_results:
    print(f"✅ Found {len(range_results)} CRP opportunities across {len(set(r['scan_date'] for r in range_results))} days")
else:
    print("⚠️  No CRP patterns found in the date range")

In [ ]:
# Display CRP Results Table
if range_results:
    scanner.display_results_table(range_results, "CRP Scanner Demo Results")
else:
    print("⚠️  No results to display")

In [ ]:
# Export Results to CSV
if range_results:
    export_filename = f"crp_scanner_demo_{date.today().strftime('%Y%m%d')}.csv"
    scanner.export_results(range_results, export_filename)
    print(f"💾 Results exported to: {export_filename}")
else:
    print("⚠️  No results to export")

In [ ]:
# CRP Pattern Analysis
if range_results:
    print("\n📊 CRP Pattern Analysis:")
    
    # Calculate statistics
    total_patterns = len(range_results)
    successful_patterns = sum(1 for r in range_results if r.get('crp_successful', False))
    success_rate = (successful_patterns / total_patterns * 100) if total_patterns > 0 else 0
    avg_probability = sum(r.get('crp_probability_score', 0) for r in range_results) / total_patterns if total_patterns > 0 else 0
    avg_price_change = sum(r.get('price_change_pct', 0) for r in range_results) / total_patterns if total_patterns > 0 else 0
    
    print(f"   Total CRP Patterns: {total_patterns}")
    print(f"   Successful Patterns: {successful_patterns}")
    print(f"   Success Rate: {success_rate:.1f}%")
    print(f"   Average Probability Score: {avg_probability:.1f}%")
    print(f"   Average Price Change: {avg_price_change:.2f}%")
    
    # Close position distribution
    close_positions = {}
    for r in range_results:
        pos = r.get('close_position', 'Unknown')
        close_positions[pos] = close_positions.get(pos, 0) + 1
    
    print(f"\n   Close Position Distribution:")
    for pos, count in close_positions.items():
        print(f"   {pos}: {count} patterns ({count/total_patterns*100:.1f}%)")
else:
    print("⚠️  No pattern analysis available")

## 🎯 CRP Pattern Explanation

### C - Close Position
- **Near High**: Stock closed within 2% of its daily high
- **Near Low**: Stock closed within 2% of its daily low  
- **Mid Range**: Stock closed in the middle of its trading range

### R - Range Tightness
- **Tight Range**: Daily range under 3% (consolidation)
- **Volume Confirmation**: Adequate trading volume

### P - Pattern Scoring
- **Probability Score**: Weighted combination of all factors (0-100)
- **Higher scores** = Better pattern quality
- **Success Rate**: Percentage of patterns that moved higher

## 💡 Usage Tips
- Focus on patterns with **probability scores > 70%**
- **Near High** patterns often indicate bullish continuation
- **Near Low** patterns may signal potential reversals
- Always check **volume confirmation**
- Use in conjunction with other technical indicators

In [ ]:
# Show CRP Examples
print("📚 CRP Pattern Examples:")
print("=" * 50)

examples = [
    {
        'scenario': 'High Probability CRP',
        'close_position': 'Near High',
        'range_pct': 1.2,
        'volume_score': 'High',
        'momentum': 'Positive',
        'probability': 95.0
    },
    {
        'scenario': 'Medium Probability CRP',
        'close_position': 'Near Low',
        'range_pct': 2.5,
        'volume_score': 'Medium',
        'momentum': 'Neutral',
        'probability': 75.0
    },
    {
        'scenario': 'Low Probability CRP',
        'close_position': 'Mid Range',
        'range_pct': 4.8,
        'volume_score': 'Low',
        'momentum': 'Negative',
        'probability': 45.0
    }
]

for i, example in enumerate(examples, 1):
    print(f"\n{i}. {example['scenario']}")
    print(f"   Close Position: {example['close_position']}")
    print(f"   Range: {example['range_pct']}%")
    print(f"   Volume: {example['volume_score']}")
    print(f"   Momentum: {example['momentum']}")
    print(f"   CRP Probability: {example['probability']}%")

print("\n🎯 CRP Scanner Demo Complete!")
print("💡 Use this scanner to identify high-probability intraday trading opportunities.")